In [169]:
import requests
import re
import time
import numpy as np
from datetime import timedelta
from crashsimilarity.downloader import SocorroDownloader

In [7]:
BASE_URL = 'https://bugzilla.mozilla.org/rest/bug?'
URL_LIST = BASE_URL + 'quicksearch=cf_crash_signature:' + '&limit=100'
# URL_LIST = BASE_URL + 'quicksearch=\\"f1=cf_crash_signature&o1=isnotempty&resolution=---&query_format=advanced&bug_status=NEW&bug_status=ASSIGNED&bug_status=REOPENED&bug_status=RESOLVED&bug_status=VERIFIED&bug_status=CLOSED&product=Firefox\\"'

In [26]:
url = 'https://bugzilla.mozilla.org/rest/bug?product=Firefox&version=51 branch&limit=1000'

In [49]:
url1 = 'https://bugzilla.mozilla.org/rest/bug?include_fields=id,summary,status,cf_crash_signature,see_also&f1=cf_crash_signature&f2=see_also&o1=isnotempty&o2=isnotempty&product=Core&product=Firefox'

In [50]:
r1 = requests.get(url1)
r1

<Response [200]>

In [54]:
data = r1.json()
bugs = data['bugs']
len(bugs)

10000

In [52]:
r2 = requests.get(url1+'&limit=1000&offset=10000')
data = r2.json()

In [53]:
len(data['bugs'])

1000

In [55]:
see_also = [i for i in bugs if i['see_also']]
len(see_also)

133

In [160]:
def clean_signatures(sigs):
    sigs = [re.sub(r'\(.*\)', '', s.strip('[] ')) for s in sigs.split('\r\n')]
    sigs = [s[2:] if s.startswith('@ ') else s for s in sigs]
    sigs = [s.strip() for s in sigs]
    return list(set(sigs))
multiple_sig = [i for i in bugs if len(clean_signatures(i['cf_crash_signature'])) > 1]
for sig in multiple_sig:
    sig['clean'] = clean_signatures(sig['cf_crash_signature'])
len(multiple_sig)

686

In [170]:
time.time()

1494376090.2972512

In [97]:
multiple_sig.sort(key=lambda x: len(x['clean']), reverse=True)

In [171]:
traces = []
prev_t = time.time()
for i, bug in enumerate(multiple_sig):
    if i % 100 == 0:
        print('{}, time spent for last 100: {} (s)'.format(i, time.time() - prev_t))
        prev_t = time.time()
    cur = []
    for sig in bug['clean']:
        try:
            r = list(SocorroDownloader().download_stack_traces_for_signature(sig, traces_num=1, period=timedelta(days=350)))[0]
            if r:
                cur.append(r)
        except:
            pass
    traces.append(cur)

0, time spent for last 100: 0.00019550323486328125 (s)
100, time spent for last 100: 318.5518651008606 (s)
200, time spent for last 100: 400.7205352783203 (s)
300, time spent for last 100: 450.31249618530273 (s)
400, time spent for last 100: 383.8052191734314 (s)
500, time spent for last 100: 537.849912405014 (s)
600, time spent for last 100: 615.417822599411 (s)


In [178]:
sorted_traces = sorted(traces, key = lambda x: len(x), reverse=True)

In [181]:
len(sorted_traces[100])

2

In [168]:
[len(i) for i in traces]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [162]:
xx = list(SocorroDownloader().download_stack_traces_for_signature('nsBoxFrame::AttributeChanged',traces_num=1, period=timedelta(days=350)))

In [164]:
xx[0]

'nsBoxFrame::AttributeChanged | mozilla::RestyleManager::AttributeChanged | PresShell::AttributeChanged | mozilla::dom::Element::SetAttrAndNotify | mozilla::dom::Element::SetAttr | mozilla::dom::Element::SetAttribute | mozilla::dom::Element::GetAttribute | mozilla::dom::ElementBinding::setAttribute | mozilla::dom::ElementBinding::hasAttribute | js::GetProperty'

In [161]:
[(i['clean']) for i in multiple_sig[:10]]

[['0x00000000 - nsDSURIContentListener::IsPreferred',
  'nsDSURIContentListener::IsPreferred'],
 ['nsCSSFrameConstructor::ConstructDocElementFrame',
  '.__ptr_glue - nsCSSFrameConstructor::ContentInserted'],
 ['0x00000000 - nsFrameList::DestroyFrames', 'nsLineLayout::ReflowFrame'],
 ['nsTableRowFrame::IR_TargetIsChild', 'nsBlockFrame::ComputeFinalSize'],
 ['nsHttpConnection::Init', 'nsHttpHandler::InitiateTransaction_Locked'],
 ['SearchTable', 'AtomKey_base::HashCode'],
 ['nsBoxFrame::AttributeChanged', 'nsMenuPopupFrame::RelayoutDirtyChild'],
 ['nsEventStateManager::DispatchMouseEvent',
  '0x00000960 | 0x00000780 | 0x00000000'],
 ['nsXBLInsertionPoint::GetDefaultContent',
  'nsXBLPrototypeHandler::GetEventName'],
 ['@0x0 | nsRuleNode::Transition',
  'nsRefPtr<imgRequest>::operator=',
  'nsRefPtr<T>::operator= | nsRuleNode::Transition']]

In [145]:
traces = SocorroDownloader().download_stack_traces_for_signature(multiple_sig[0]['clean'][1], period=timedelta(days=350))
traces

{'@0x12f6a2 | nsDSURIContentListener::IsPreferred | nsDocumentOpenInfo::TryContentListener | PR_Assert | nsDocumentOpenInfo::OnStartRequest | mozilla::net::nsHttpChannel::ContinueOnStartRequest2',
 'nsDSURIContentListener::IsPreferred | nsDocumentOpenInfo::TryContentListener | mozilla::net::HttpBaseChannel::GetContentDisposition | nsDocumentOpenInfo::DispatchContent'}

In [136]:
multiple_sig[0]

{'cf_crash_signature': '[@ nsDSURIContentListener::IsPreferred]\r\n[@ 0x00000000 - nsDSURIContentListener::IsPreferred]',
 'clean': ['0x00000000 - nsDSURIContentListener::IsPreferred',
  'nsDSURIContentListener::IsPreferred'],
 'id': 67721,
 'see_also': [],
 'status': 'VERIFIED',
 'summary': 'Trunk M096 N621 - Win2K/WinNT crash [@ nsDSURIContentListener::IsPreferred] [@ 0x00000000 - nsDSURIContentListener::IsPreferred]'}

In [106]:
see_also = [i for i in see_also if any(['bugzilla' in s for s in i['see_also']])]
len(see_also)

78

In [107]:
see_also[:10]

[{'cf_crash_signature': '[@ InlineBackgroundData::GetContinuousRect]',
  'id': 421203,
  'see_also': ['https://bugzilla.mozilla.org/show_bug.cgi?id=1209896'],
  'status': 'VERIFIED',
  'summary': 'Crash [@ InlineBackgroundData::GetContinuousRect] with XUL, rtl, background image'},
 {'cf_crash_signature': '[@ gtk_widget_get_toplevel]',
  'id': 444055,
  'see_also': ['https://bugzilla.gnome.org/show_bug.cgi?id=494542'],
  'status': 'RESOLVED',
  'summary': 'browser crashes when a file is downloaded [@ gtk_widget_get_toplevel]'},
 {'cf_crash_signature': '[@ nsRDFPropertyTestNode::FilterInstantiations]',
  'id': 461695,
  'see_also': ['https://bugzilla.mozilla.org/show_bug.cgi?id=1057581'],
  'status': 'NEW',
  'summary': '[@ nsRDFPropertyTestNode::FilterInstantiations]'},
 {'cf_crash_signature': '[@XPCWrappedNative::GetNewOrUsed]',
  'id': 487619,
  'see_also': ['https://bugzilla.mozilla.org/show_bug.cgi?id=603075'],
  'status': 'RESOLVED',
  'summary': 'Crash [@XPCWrappedNative::GetNewOr

In [74]:
clean_signatures(multiple_sig[999]['cf_crash_signature'])

['[@ js::types::TypeCompartment::markSetsUnknown ]']

In [65]:
multiple_sig[999]

{'cf_crash_signature': '[@ js::types::TypeCompartment::markSetsUnknown(JSContext*, js::types::TypeObject*) ]\r\n[@ js::types::TypeCompartment::markSetsUnknown ]',
 'id': 714625,
 'see_also': [],
 'status': 'NEW',
 'summary': 'crash in js::types::TypeCompartment::markSetsUnknown'}

In [36]:
url2 = url1 + '&limit=10'
r2 = requests.get(url2)
r2

<Response [200]>

In [27]:
url3 = url2+'&offset=20'
r3 = requests.get(url3)
r3

<Response [200]>

In [37]:
r2.json()['bugs']

[{'cf_crash_signature': '[@ ns4xPlugin::_memalloc]',
  'id': 3391,
  'see_also': [],
  'status': 'VERIFIED',
  'summary': 'Cosmoplayer causes a crash [@ ns4xPlugin::_memalloc]'},
 {'id': 28358,
  'see_also': [],
  'status': 'RESOLVED',
  'summary': 'Crash triggering when (folder) object inserted in AddFile, AddDir, or Patch version string [@ ConvertJSvalToVersionString]'},
 {'cf_crash_signature': '[@ js_LockScope1]\n[@ js_GetSlotWhileLocked]',
  'id': 31847,
  'see_also': [],
  'status': 'RESOLVED',
  'summary': 'Need to solve problem of JS roots for event handler funcs [@ js_LockScope1] [@ js_GetSlotWhileLocked]'},
 {'id': 33604,
  'see_also': [],
  'status': 'VERIFIED',
  'summary': 'N601 Crash (Mac) #2: Browser crashes on loading applet after creating new profile [@ OTClientLib + 0x85c]'},
 {'cf_crash_signature': '[@ nsView::GetDimensions ]',
  'id': 35299,
  'see_also': [],
  'status': 'RESOLVED',
  'summary': 'textfield inside menupopup does not behave, causes crash [@ nsView::Get

In [33]:
bugs[20:30] == r3.json()['bugs']

True

In [50]:
type(data)

list

In [29]:
with_sig = [i for i in data if i['cf_crash_signature']]

In [31]:
with_sig

[{'alias': None,
  'assigned_to': 'nobody@mozilla.org',
  'assigned_to_detail': {'email': 'nobody@mozilla.org',
   'id': 1,
   'name': 'nobody@mozilla.org',
   'real_name': 'Nobody; OK to take it and work on it'},
  'blocks': [],
  'cc': ['aklotz@mozilla.com',
   'gijskruitbosch+bugs@gmail.com',
   'mcastelluccio@mozilla.com'],
  'cc_detail': [{'email': 'aklotz@mozilla.com',
    'id': 454100,
    'name': 'aklotz@mozilla.com',
    'real_name': 'Aaron Klotz [:aklotz] (a11y work receiving priority right now)'},
   {'email': 'gijskruitbosch+bugs@gmail.com',
    'id': 159069,
    'name': 'gijskruitbosch+bugs@gmail.com',
    'real_name': ':Gijs'},
   {'email': 'mcastelluccio@mozilla.com',
    'id': 420453,
    'name': 'mcastelluccio@mozilla.com',
    'real_name': 'Marco Castelluccio [:marco]'}],
  'cf_crash_signature': '[@ NotifyInitializeSpies]',
  'cf_fx_iteration': '---',
  'cf_fx_points': '---',
  'cf_has_regression_range': '---',
  'cf_has_str': '---',
  'cf_last_resolved': '2016-10-13T